<a href="https://colab.research.google.com/github/iam-fern/Gen_datanew/blob/main/GQtest_cluster_loop_gendatanew_n20_x4_1000rep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
df = pd.read_csv('https://github.com/iam-fern/Gen_datanew/raw/main/gendatanew_n20_x4_1000rep.csv', index_col=['rep', 'index'])
df

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    62.062241  96.471382  ...  192944.346176 -1.241821e+07
      1.0    28.074258  44.070832  ...  -61192.497829  1.195692e+06
      2.0    45.948494  46.610042  ... -229453.091911  9.023657e+06
      3.0    27.407607  42.034405  ... -104106.307251  3.422525e+05
      4.0    71.951060  35.733543  ... -599212.929043 -2.672621e+07
...                ...        ...  ...            ...           ...
999.0 45.0   33.982262  79.866862  ...  304206.051288  4.810935e+05
      46.0   90.476397  90.833758  ... -164227.249816  4.301619e+07
      47.0   71.582816  46.313929  ... -485059.717133  8.206874e+06
      48.0   72.249787  41.227621  ... -534513.919767  2.480777e+07
      49.0   80.343196  84.300220  ... -116674.990495  3.534228e+07

[50000 rows x 7 columns]

In [10]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    62.062241  96.471382  ...  192944.346176 -1.241821e+07
      1.0    28.074258  44.070832  ...  -61192.497829  1.195692e+06
      2.0    45.948494  46.610042  ... -229453.091911  9.023657e+06
      3.0    27.407607  42.034405  ... -104106.307251  3.422525e+05
      4.0    71.951060  35.733543  ... -599212.929043 -2.672621e+07
...                ...        ...  ...            ...           ...
999.0 45.0   33.982262  79.866862  ...  304206.051288  4.810935e+05
      46.0   90.476397  90.833758  ... -164227.249816  4.301619e+07
      47.0   71.582816  46.313929  ... -485059.717133  8.206874e+06
      48.0   72.249787  41.227621  ... -534513.919767  2.480777e+07
      49.0   80.343196  84.300220  ... -116674.990495  3.534228e+07

[50000 rows x 7 columns]

In [11]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,62.062241,96.471382,43.954380,-1.222548e+07,-1.222527e+07,192944.346176,-1.241821e+07
1,0.0,1.0,28.074258,44.070832,80.998193,1.134337e+06,1.134500e+06,-61192.497829,1.195692e+06
2,0.0,2.0,45.948494,46.610042,36.547974,8.794065e+06,8.794204e+06,-229453.091911,9.023657e+06
3,0.0,3.0,27.407607,42.034405,25.623708,2.380412e+05,2.381462e+05,-104106.307251,3.422525e+05
4,0.0,4.0,71.951060,35.733543,66.105808,-2.732560e+07,-2.732542e+07,-599212.929043,-2.672621e+07
...,...,...,...,...,...,...,...,...,...
49995,999.0,45.0,33.982262,79.866862,99.414090,7.850763e+05,7.852996e+05,304206.051288,4.810935e+05
49996,999.0,46.0,90.476397,90.833758,21.500823,4.285175e+07,4.285196e+07,-164227.249816,4.301619e+07
49997,999.0,47.0,71.582816,46.313929,39.933602,7.721647e+06,7.721814e+06,-485059.717133,8.206874e+06
49998,999.0,48.0,72.249787,41.227621,74.488343,2.427306e+07,2.427325e+07,-534513.919767,2.480777e+07


In [12]:
table.set_index(['rep', 'index'], inplace=True)

In [13]:
table

x1         x2  ...          y_hat         e_hat
rep   index                        ...                             
0.0   0.0    62.062241  96.471382  ...  192944.346176 -1.241821e+07
      1.0    28.074258  44.070832  ...  -61192.497829  1.195692e+06
      2.0    45.948494  46.610042  ... -229453.091911  9.023657e+06
      3.0    27.407607  42.034405  ... -104106.307251  3.422525e+05
      4.0    71.951060  35.733543  ... -599212.929043 -2.672621e+07
...                ...        ...  ...            ...           ...
999.0 45.0   33.982262  79.866862  ...  304206.051288  4.810935e+05
      46.0   90.476397  90.833758  ... -164227.249816  4.301619e+07
      47.0   71.582816  46.313929  ... -485059.717133  8.206874e+06
      48.0   72.249787  41.227621  ... -534513.919767  2.480777e+07
      49.0   80.343196  84.300220  ... -116674.990495  3.534228e+07

[50000 rows x 7 columns]

In [14]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)
    data_sort_clusters = data_clusters.sort_values('clusters')
    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid + 1
    SSR1 = model1.ssr
    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid + 1
    SSR0 = model0.ssr
    MSE0 = model0.mse_resid

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/DF1) / (SSR0/DF0)

    df1 = DF1
    df0 = DF0

    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [15]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,18.0,26.0,1.4489212565567134e+16,2.4384659708955588e+16,852306621503949.0,975386388358223.5,1.1651211143753848,0.7488289233993899,0.625585538300305,Reject005=1 : Homoscedasticity
2.0,20.0,24.0,1.3753561294889154e+16,6.736404829073515e+16,723871647099429.1,2928871664814572.0,4.081612442430198,0.0022329294825469415,0.9988835352587265,Reject005=0 : Heteroscedasticity
8.0,24.0,20.0,430921858966849.8,5.761211083825641e+16,18735732998558.688,3032216359908232.0,160.43403593324356,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
10.0,27.0,17.0,3357325895653740.5,5.6750674885538584e+16,129127919063605.4,3546917180346161.5,26.846790457516043,2.5719426588466376e-12,0.999999999998714,Reject005=0 : Heteroscedasticity
11.0,18.0,26.0,6504888846130655.0,2.766716529935493e+16,382640520360626.75,1106686611974197.1,2.944583960490806,0.020878436491147623,0.9895607817544262,Reject005=0 : Heteroscedasticity
16.0,20.0,24.0,1344666607929238.2,6.725736135626083e+16,70771926733117.8,2924233102446123.0,41.68155942871961,4.846123502488808e-12,0.9999999999975769,Reject005=0 : Heteroscedasticity
19.0,21.0,23.0,8102671060542075.0,3.3150027564542132e+16,405133553027103.75,1506819434751915.0,3.7354862669136093,0.0034990773488499105,0.998250461325575,Reject005=0 : Heteroscedasticity
20.0,16.0,28.0,4926548794449151.0,5.7788653268223e+16,328436586296610.06,2140320491415666.8,6.7028845058935405,0.00023161093905943453,0.9998841945304703,Reject005=0 : Heteroscedasticity
22.0,21.0,23.0,1.0286299740383564e+16,1.375107766922411e+16,514314987019178.2,625048984964732.2,1.2205877819846205,0.6497296311110117,0.6751351844444942,Reject005=1 : Homoscedasticity


In [16]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,18.0,26.0,1.4489212565567134e+16,2.4384659708955588e+16,852306621503949.0,975386388358223.5,1.1651211143753848,1.25117107660061,Reject005=1 : Homoscedasticity
1.0,22.0,22.0,9.15102885638114e+16,2.8047592073172444e+16,4357632788752923.5,1335599622532021.2,0.3064965974139014,0.007614561140408919,Reject005=0 : Heteroscedasticity
2.0,20.0,24.0,1.3753561294889154e+16,6.736404829073515e+16,723871647099429.1,2928871664814572.0,4.081612442430198,0.0022329294825469415,Reject005=0 : Heteroscedasticity
3.0,29.0,15.0,5.8515815203909736e+16,1.7765384365288172e+16,2089850542996776.2,1268956026092012.2,0.5869594340129027,0.2778807141532838,Reject005=1 : Homoscedasticity
4.0,21.0,23.0,2.833030971120277e+16,1969234631065746.2,1416515485560138.5,89510665048443.02,0.0634654846836722,8.18104947819285e-09,Reject005=0 : Heteroscedasticity
5.0,25.0,19.0,1.1596486396991646e+16,4994047928172274.0,483186933207985.25,277447107120681.88,0.566647126552763,0.20750585007515981,Reject005=1 : Homoscedasticity
6.0,17.0,27.0,2.987766658737277e+16,3613641379525924.0,1867354161710798.0,138986206904843.23,0.07615238883369846,1.259500026767507e-08,Reject005=0 : Heteroscedasticity
7.0,18.0,26.0,2.834572142788094e+16,1.1142051082527796e+16,1667395378110643.5,445682043301111.8,0.2721302293238512,0.0026243195156101552,Reject005=0 : Heteroscedasticity
8.0,24.0,20.0,430921858966849.8,5.761211083825641e+16,18735732998558.688,3032216359908232.0,160.43403593324356,2.220446049250313e-16,Reject005=0 : Heteroscedasticity


In [17]:
#two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 1 if p_value < alpha else 0)
#two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
#two_tailed_new_show.astype(str)

In [18]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    658
Reject005=1 : Homoscedasticity      342
Name: Result_test, dtype: int64